# Predicting FWI using deepFWI Pre-trained models

### Introduction
The Fire Weather Index (FWI) is a meteorologically based index used worldwide to estimate fire danger. It consists of different components that account for the effects of fuel moisture and wind on fire behaviour and spread. The higher the FWI, the more favorable the meteorological conditions to trigger a wildfire. This indicator can help shape long-term tourist strategy and to plan future investments under a changing climate.

The [`esowc/wildfire-forecasting`](https://github.com/esowc/wildfire-forecasting) project intends to reproduce the Fire Forecasting capabilities of GEFF using Deep Learning and develop further improvements in accuracy, geography and time scale through inclusion of additional variables or optimisation of model architecture & hyperparameters. 

In this notebook we demonstrate the use of pre-trained Deep Learning models to perform FWI inference. The Deep Learning Model has been trained on 1 year of data and takes as input the four weather forcings used in the numerical calculation of FWI:
```
* t2    -    Temperature at 2m
* tp    -    Total Precipitation
* rh    -    Relative Humidity
* wspeed-    Windspeed
* swvl1 -    Volumetric Soil Water Level 1
```

These input weather forcings are supplied for 4 days and the model produces FWI forecast for 14 days. There is 1 day of overlap between the 4 weather forcing variable inputs and the output FWI forecast as indicated below:
```
                                               ____________
Weather Forcings + SWVL1 (t-3, t-2, t-1, t) -> | deepFWI  |  -> deepFWI-Forecast (t, t+1, t+2, ..., t+13)
                                               ------------
```

To run this notebook, make sure you have the `wildfire-dl` `conda` environment activated. Instructions for setting up the environment are available in the project [`README`](https://github.com/esowc/wildfire-forecasting/blob/master/README.md).

### Getting the data

We first obtain a sample dataset. This is the minimum data that we need to perform inference and then benchmark the results. This data is stored on a publicly accessible Google Cloud Platform (GCP) storage bucket at `gs://deepgeff-data-v0/` and is downloaded into sampledata/

In [1]:
import os
from IPython.display import Image
os.chdir('..')

### Ensure access to the data
This example will require the following dataset from `gs://deepgeff-data-v0/` -
* `fwi-forcings/`: Weather Forcings + SKT for 2019-12-01 to 2019-12-04 --> 4 days --> 4 files 
* `fwi-reanalysis/`: ERA Reanalysis of FWI for 2019-12-04 to 2019-12-17 --> 14 days --> 14 files

If you wish to explore these datasets further, have a look at the EDA notebooks in `data/EDA/` in the repository.

In [2]:
# Uncomment to download a minimal sample dataset consisting of Weather Forcings & SWVL1
# for 2019-12-01 to 2019-12-04 --> 4 days --> 4 files
!mkdir sampledata/4_14/forcings -p
!gsutil -m -q cp -r 'gs://deepgeff-data-v0/forcings/ECMWF_FO_2019120[1-4]*' sampledata/4_14/forcings
!gsutil -m -q cp -r 'gs://deepgeff-data-v0/swvl1/ECMWF_FO_2019120[1-4]*' sampledata/4_14/forcings

In [3]:
# Uncomment to download a minimal sample dataset consisting of ERA Reanalysis 
# for 2019-12-04 to 2019-12-17 --> 14 days --> 14 files
!mkdir sampledata/4_14/fwi-reanalysis -p
!gsutil -m -q cp -r 'gs://deepgeff-data-v0/fwi-reanalysis/ECMWF_FWI_201912[0][4-9]*' sampledata/4_14/fwi-reanalysis
!gsutil -m -q cp -r 'gs://deepgeff-data-v0/fwi-reanalysis/ECMWF_FWI_201912[1][0-7]*' sampledata/4_14/fwi-reanalysis

### Getting the pre-trained model

For this example notebook, we use the following model-

* Run ID: `f1nbkzcf` that has been trained for 30 epochs on 4 months of T2, TP, RH, WSpeed and SWVL1 data for Aug 2019 - Nov, 2019.

If you wish to explore other models, have a look at pre-trained_models.md.

In [4]:
# Uncomment to download a sample pre-trained model checkpoint file
!mkdir samplemodel/4_14/ -p
!gsutil -q cp 'gs://deepgeff-models-v0/pre-trained_models/4_14/f1nbkzcf/f1nbkzcf_28_30.ckpt' samplemodel/4_14/

### Running the inference and the benchmark

For performing inference and then benchmarking the results, we use the `run(**kwargs)` function from [`src/test.py`](https://github.com/esowc/wildfire-forecasting/blob/master/src/test.py) which takes as argument a dict of configuration parameters and boolean for `benchmark`. You can check out more details about this function in [`test.py#L266`](https://github.com/esowc/wildfire-forecasting/blob/master/src/test.py#L266)

Providing the configuration options,

```
* in_days        -    no of input days of weather forcings
* out_days       -    no of output days of fwi prediction
* checkpoint_file-    path to pretrained checkpoint file
* reanalysis_dir -    path to fwi-reanalysis data
* forcings_dir   -    path to fwi-forcings data
* forecast_dir   -    path to fwi-forecast data
```

In [5]:
!python src/test.py -in-days 4 -out-days 14 -checkpoint-file "samplemodel/4_14/f1nbkzcf_28_30.ckpt" \
-forcings-dir "sampledata/4_14/forcings" -reanalysis-dir "sampledata/4_14/fwi-reanalysis"

 |       init-features -> 20                  
 |             in-days -> 4                   
 |            out-days -> 14                  
 |              epochs -> 100                 
 |       learning-rate -> 0.001               
 |          batch-size -> 1                   
 |               split -> 0.2                 
 |           use-16bit -> True                
 |                gpus -> 1                   
 |               optim -> one_cycle           
 |             dry-run -> False               
 |             find-lr -> False               
 |           search-bs -> False               
 |          case-study -> False               
 |         clip-output -> False               
 |              boxcox -> 0.1182              
 |              binned -> [0, 5.2, 11.2, 21.3, 38.0, 50]
 |       round-to-zero -> False               
 |         isolate-frp -> False               
 |          date-range -> False               
 |             cb-loss -> False               
 | 

 'mae_50_inf_6': tensor(60.2308, device='cuda:0'),
 'mae_50_inf_7': tensor(59.3697, device='cuda:0'),
 'mae_50_inf_8': tensor(59.1039, device='cuda:0'),
 'mae_50_inf_9': tensor(52.9584, device='cuda:0'),
 'mae_6': tensor(29.9397, device='cuda:0'),
 'mae_7': tensor(12.5855, device='cuda:0'),
 'mae_8': tensor(13.5020, device='cuda:0'),
 'mae_9': tensor(13.5738, device='cuda:0'),
 'mse': tensor(118850.9688, device='cuda:0'),
 'mse_0': tensor(467.8320, device='cuda:0'),
 'mse_0_5.2_0': tensor(153.2978, device='cuda:0'),
 'mse_0_5.2_1': tensor(219.5451, device='cuda:0'),
 'mse_0_5.2_10': tensor(117.8066, device='cuda:0'),
 'mse_0_5.2_11': tensor(96.6988, device='cuda:0'),
 'mse_0_5.2_12': tensor(119.8932, device='cuda:0'),
 'mse_0_5.2_13': tensor(271.6765, device='cuda:0'),
 'mse_0_5.2_2': tensor(444.8809, device='cuda:0'),
 'mse_0_5.2_3': tensor(255152.6562, device='cuda:0'),
 'mse_0_5.2_4': tensor(3024.4585, device='cuda:0'),
 'mse_0_5.2_5': tensor(1286770.8750, device='cuda:0'),
 'mse_0_

### Results

We demonstrate below the metrics (accuracy, mean-squared-error and mean-absolute-error) for deepFWI predicted values. These metrics are for 1 FWI prediction (for 14 days) in a global setting. The metrics are calculated against fwi-reanalysis (considered the ground truth). The grouped bar plots indicate the metrics for deepFWI while the line plots are used for fwi-forecast. The FWI values are binned as per the EFFIS categorisation:

```
* [0, 5.2]     - Very Low
* (5.2, 11.2]  - Low
* (11.2, 21.3] - Moderate
* (21.3, 38.0] - High
* (38.0, 50.0] - Very High
* (50.0, Inf)  - Extreme
```


In [6]:
# Generate result plots
!python src/plot.py -f result.json -i 4 -o 14
!ls acc* mae* mse*

acc_20201212-070254.png  mae_20201212-070303.png  mse_20201212-070259.png


In [7]:
Image(filename='acc_20201212-070254.png')